In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from iteration_utilities import random_combination
from sklearn.cluster import KMeans
from tqdm import tqdm
import json

from models.baseline_model import ArticleBaselineModel
from models.tabularncd_model import TabularNCDModel
from models.NCD_Spectral_Clustering import *
from models.NCD_Kmeans import k_means_pp
from models.PBN_model import PBNModel
from src.dataset_utils import *
from src.utils import *

In [ ]:
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe'  # So we can view the plots in the jupyter notebook

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = setup_device(use_cuda=True)

In [ ]:
def create_known_unknown_classes_splits(all_classes, n_unknown_classes, k_folds, random_seed=None):
    """
    Select :k_folds: random combinations of :n_unknown_classes: from the given :all_classes:.
    :param all_classes: the unique known classes where the probe classes will be selected from.
    :param n_unknown_classes: ToDo.
    :param k_folds: ToDo.
    :param random_seed: ToDo.
    """
    n_classes = len(all_classes)
    number_of_possible_combinations = math.comb(n_classes, min(n_unknown_classes, n_classes))
    
    if k_folds == 0:
        print(f"k_folds set to 0, using the maximum number of possible combinations: {number_of_possible_combinations}")
        k_folds = number_of_possible_combinations
    
    # Sanity checks
    if n_unknown_classes >= n_classes:
        raise ValueError(f"There are only {n_classes} known classes, cannot use {n_unknown_classes} classes as probe set for cross validation.")
    if k_folds > number_of_possible_combinations:
        raise ValueError(f"Not enough classes to have {k_folds} folds given {n_unknown_classes} unknown classes and {n_classes} total classes, the maximum is {number_of_possible_combinations}.")
    
    if random_seed is not None:
        random.seed(random_seed)
    
    # Select random combinations of unknown classes
    selected_combinations = []
    while len(selected_combinations) < k_folds:
        new_combination = random_combination(all_classes, n_unknown_classes)
        if new_combination not in selected_combinations:
            selected_combinations.append(new_combination)
            
    # From these selected lists of unknown classes, create the known/unknown classes splits
    known_classes_splits = []
    unknown_classes_splits = []
    for combination in selected_combinations:
        unknown_classes_splits.append(list(combination))
        known_classes_splits.append([known_class for known_class in all_classes if known_class not in list(combination)])
    
    return known_classes_splits, unknown_classes_splits

In [ ]:
def joint_shuffling(x, y, z=None):
    np.random.seed(0)
    p = np.random.permutation(len(x))
    
    if z is None:
        return x[p], y[p], None
    else:
        return x[p], y[p], z[p]

In [ ]:
dataset_names = ['HumanActivityRecognition', 'LetterRecognition', 'Pendigits', 'USCensus1990', 'multiple_feature', 'optdigits', 'cnae_9']

dataset_name = 'USCensus1990'

original_x_train, original_y_train, original_x_test, original_y_test = import_train_and_test_datasets(dataset_name)

if dataset_name == 'USCensus1990':
    # Too much train and test data, so we cap and balance them 
    train_indices_to_keep = []
    for c in np.unique(original_y_train):
        train_indices_to_keep += list(np.arange(len(original_y_train))[original_y_train == c][:1000])
    train_indices_to_keep.sort()
    original_x_train = original_x_train[train_indices_to_keep]
    original_y_train = original_y_train[train_indices_to_keep]

    test_indices_to_keep = []
    for c in np.unique(original_y_test):
        test_indices_to_keep += list(np.arange(len(original_y_test))[original_y_test == c][:1000])
    test_indices_to_keep.sort()
    original_x_test = original_x_test[test_indices_to_keep]
    original_y_test = original_y_test[test_indices_to_keep]

In [ ]:
def get_kmeans_pred(n_clusters, x_train_unknown, x_test_unknown):
    km = KMeans(n_clusters=n_clusters, init='random', n_init=10).fit(x_train_unknown.cpu().numpy())
    return km.predict(x_test_unknown.cpu().numpy())

In [ ]:
def get_ncdkmeans_pred(n_clusters, x_train_known, y_train_known, x_train_unknown, x_test_unknown,):
    # For this method, we define the centroids of the known classes with ground truth as initial centroids
    known_classes_centroids = torch.stack([x_train_known[y_train_known==c].mean(axis=0) for c in np.unique(y_train_known)])
    kmpp = k_means_pp(pre_centroids=known_classes_centroids, k_new_centroids=n_clusters)
    kmpp.fit(x_train_unknown, tolerance=1e-4, n_iterations=300)
    return kmpp.predict_unknown_data(x_test_unknown).cpu().numpy()

In [ ]:
def get_sc_pred(n_new_clusters, x_test_unknown):
    sc = ncd_spectral_clustering(n_new_clusters=n_new_clusters, min_dist=0.6)
    return sc.fit_predict_simple(x_test_unknown)

In [ ]:
def get_ncdsc_pred(n_clusters, test_unknown_spectral_embedding):
    kmpp = k_means_pp(pre_centroids=None, k_new_centroids=n_clusters)
    kmpp.fit(test_unknown_spectral_embedding, tolerance=1e-10, n_iterations=1000, n_init=10)
    return kmpp.predict_unknown_data(test_unknown_spectral_embedding).cpu().numpy()

In [ ]:
def get_baseline_pred(n_clusters, x_train_known, y_train_known, x_test_unknown, y_test_unknown, x_test_known, y_test_known):
    base_config = {
        'input_size': x_train_known.shape[1],
        'hidden_layers_dims': [math.floor(3*x_train_known.shape[1]/4), math.floor(2*x_train_known.shape[1]/4)],
        'activation_fct': 'relu',  # relu or sigmoid or tanh or None
        'use_batchnorm': True,  # True or False
        'use_norm': 'l2',  # None or 'l1' or 'l2'
        'n_classes': len(np.unique(y_train_known)),
        'n_clusters': n_clusters,
        'clustering_model': 'kmeans',  # kmeans or ncd_kmeans or spectral_clustering or ncd_spectral_clustering
        'clustering_runs': 1,  # To compute the average accuracy of the clustering
        'batch_size': 512,
        'epochs': 200,
    }

    # We load the hyperparameters that were optimized through grid search
    d = json.load(open("hyperparameters.json"))
    config = d["Baseline"]["GT k"][dataset_name]
    # print("Using hyperparameters:", config)

    b_config = base_config.copy()
    b_config.update(config)

    model = ArticleBaselineModel(b_config).to(device)
    losses_dict = model.train_on_known_classes(x_train_known=x_train_known, y_train_known=y_train_known,
                                               x_test_unknown=x_test_unknown, y_test_unknown=y_test_unknown,
                                               x_test_known=x_test_known, y_test_known=y_test_known,
                                               batch_size=b_config['batch_size'], lr=b_config['lr'], epochs=b_config['epochs'], n_clusters=b_config['n_clusters'], clustering_runs=b_config['clustering_runs'],
                                               evaluate=False, disable_tqdm=True)

    model.eval()
    with torch.no_grad():
        y_test_unknown_pred = model.predict_new_data(b_config['n_clusters'], x_test_unknown)
    
    return y_test_unknown_pred

In [ ]:
def get_pbn_pred(n_clusters, x_train, y_train, unknown_class_value, x_test_unknown, y_test_unknown, x_test_known, y_test_known):
    base_config = {
        'input_size': x_train.shape[1],
        'hidden_layers_dims': [math.floor(3*x_train.shape[1]/4), math.floor(2*x_train.shape[1]/4)],
        'activation_fct': 'relu',  # relu or sigmoid or tanh or None
        'use_batchnorm': True,  # True or False
        'use_norm': 'l2',  # None or 'l1' or 'l2'
        'n_classes': len(np.unique(y_train)) - 1,
        'clustering_model': 'kmeans',  # kmeans or ncd_kmeans or spectral_clustering or ncd_spectral_clustering
        'clustering_runs': 1,  # To compute the average accuracy of the clustering
        'batch_size': 512,
        'epochs': 200,
    }

    # We load the hyperparameters that were optimized through grid search
    d = json.load(open("hyperparameters.json"))
    config = d["PBN"]["GT k"][dataset_name]
    # print("Using hyperparameters:", config)

    pbn_config = base_config.copy()
    pbn_config.update(config)

    model = PBNModel(pbn_config).to(device)
    losses_dict = model.train_on_known_classes(x_train=x_train, y_train=y_train, unknown_class_value=unknown_class_value,
                                               x_test_unknown=x_test_unknown, y_test_unknown=y_test_unknown,
                                               x_test_known=x_test_known, y_test_known=y_test_known,
                                               batch_size=pbn_config['batch_size'], lr=pbn_config['lr'], epochs=pbn_config['epochs'], clustering_runs=pbn_config['clustering_runs'], w=pbn_config['w'],
                                               evaluate=False, disable_tqdm=True)
    
    model.eval()
    with torch.no_grad():
        y_test_unknown_pred = model.predict_new_data(n_clusters=n_clusters, x_unknown=x_test_unknown)
    
    return y_test_unknown_pred

In [ ]:
def train_tabularncd(n_clusters, x_train, y_train, dataset_name, x_test_known, y_test_known, x_test_unknown, y_test_unknown, y_train_unknown, unknown_class_value):
    base_config = {
        'hidden_layers_dims': [math.floor(3*x_train.shape[1]/4), math.floor(2*x_train.shape[1]/4)],
        'input_size': x_train.shape[1],
        'n_known_classes': len(np.unique(y_train)),  # Takes into account the unknown class
        'n_unknown_classes': n_clusters,
        'activation_fct': 'relu',
        'use_batchnorm': True,
        'batch_size': 512,
        'epochs': 200,
        'M': 2000,
    }
    # We load the hyperparameters that were optimized through grid search
    d = json.load(open("hyperparameters.json"))
    config = d["TabularNCD"]["GT k"][dataset_name]
    # print("Using hyperparameters:", config)

    tncd_config = base_config.copy()
    tncd_config.update(config)

    model = TabularNCDModel(tncd_config).to(device)
    losses_dict = model.joint_training(config=tncd_config,
                                       x_train=x_train, y_train=y_train,
                                       x_test_known=x_test_known, y_test_known=y_test_known,
                                       x_test_unknown=x_test_unknown, y_test_unknown=y_test_unknown,
                                       y_train_unknown=y_train_unknown, unknown_class_value=unknown_class_value, disable_tqdm=True)
    
    model.eval()
    with torch.no_grad():
        y_test_unknown_pred = model.clustering_head_forward(model.encoder_forward(x_test_unknown)).argmax(-1).cpu().numpy()
    
    return y_test_unknown_pred

In [ ]:
def get_model_performance(model_name, n_runs,
                          x_train, y_train, unknown_class_value,
                          x_train_known, y_train_known,
                          x_train_unknown, y_train_unknown,
                          x_test_known, y_test_known,
                          x_test_unknown, y_test_unknown,
                          test_unknown_spectral_embedding, t):
    
    n_clusters = len(np.unique(y_train_unknown))
    
    accs, nmis, aris = [], [], []
        
    for i in range(n_runs):
        t.set_postfix_str("run " + str(i+1) + " / " + str(n_runs))
        
        if model_name == 'kmeans':
            y_test_unknown_pred = get_kmeans_pred(n_clusters, x_train_unknown, x_test_unknown)
        elif model_name == 'ncdkmeans':
            y_test_unknown_pred = get_ncdkmeans_pred(n_clusters, x_train_known, y_train_known, x_train_unknown, x_test_unknown)
        elif model_name == 'sc':
            y_test_unknown_pred = get_sc_pred(n_clusters, x_test_unknown)
        elif model_name == 'ncdsc':
            y_test_unknown_pred = get_ncdsc_pred(n_clusters, test_unknown_spectral_embedding)
        elif model_name == 'baseline':
            y_test_unknown_pred = get_baseline_pred(n_clusters, x_train_known, y_train_known, x_test_unknown, y_test_unknown, x_test_known, y_test_known)
        elif model_name == 'pbn':
            y_test_unknown_pred = get_pbn_pred(n_clusters, x_train, y_train, unknown_class_value, x_test_unknown, y_test_unknown, x_test_known, y_test_known)
        elif model_name == 'tabularncd':
            y_test_unknown_pred = train_tabularncd(n_clusters, x_train, y_train, dataset_name, x_test_known, y_test_known, x_test_unknown, y_test_unknown, y_train_unknown, unknown_class_value)
            
        accs.append(hungarian_accuracy(y_test_unknown, y_test_unknown_pred))
        nmis.append(normalized_mutual_info_score(y_test_unknown, y_test_unknown_pred))
        aris.append(adjusted_rand_score(y_test_unknown, y_test_unknown_pred))
        
        t.update()
        
    return accs, nmis, aris

In [ ]:
def get_model_perf_wrt_n_unknown_classes(classes_range, model_name, n_runs, k_folds):
    
    # In this case, we pre-compute the spectral embedding to avoid doing it every time
    # Since it is deterministic, it should not be a problem
    if model_name == 'ncdsc':
        # We load the hyperparameters that were optimized through grid search
        d = json.load(open("hyperparameters.json"))
        ncdsc_params = d["NCD SC"]["GT k"][dataset_name]
        # print("Using hyperparameters:", ncdsc_params)

        # Get the spectral embedding for all the data (since the hyperparameters are adapted for all the data, not only the novel data)
        full_spectral_embedding = get_spectral_embedding(torch.tensor(original_x_test, dtype=torch.float, device=device), ncdsc_params['n_components'], ncdsc_params['min_dist'])
    else:
        full_spectral_embedding = None
    
    with tqdm(total = len(classes_range) * k_folds * n_runs) as t:
    
        accs_avg_1, nmis_avg_1, aris_avg_1 = [], [], []
        accs_std_1, nmis_std_1, aris_std_1 = [], [], []
        for n_unknown_classes in classes_range:

            known_classes_splits, unknown_classes_splits = create_known_unknown_classes_splits(all_classes=np.unique(original_y_train), n_unknown_classes=n_unknown_classes, k_folds=k_folds, random_seed=0)

            accs_avg_2, nmis_avg_2, aris_avg_2 = [], [], []
            accs_std_2, nmis_std_2, aris_std_2 = [], [], []
            for known_classes_split, unknown_classes_split in zip(known_classes_splits, unknown_classes_splits):
                # print(f"known_classes_split={known_classes_split}, unknown_classes_split={unknown_classes_split}")

                x_train, y_train, _ = joint_shuffling(original_x_train, original_y_train)
                x_test, y_test, shuffled_full_spectral_embedding = joint_shuffling(original_x_test, original_y_test, full_spectral_embedding)
                x_train, y_train, x_test, y_test, unknown_class_value, y_train_save, y_test_save = select_known_unknown_classes(x_train, y_train,
                                                                                                                                x_test, y_test,
                                                                                                                                known_classes=known_classes_split,
                                                                                                                                unknown_classes=unknown_classes_split,
                                                                                                                                silent=True)
                
                x_train = torch.tensor(x_train, dtype=torch.float, device=device)
                x_test = torch.tensor(x_test, dtype=torch.float, device=device)

                x_train_known = x_train[y_train != unknown_class_value]
                x_train_unknown = x_train[y_train == unknown_class_value]
                y_train_known = y_train[y_train != unknown_class_value]

                x_test_known = x_test[y_test != unknown_class_value]
                x_test_unknown = x_test[y_test == unknown_class_value]
                y_test_known = y_test[y_test != unknown_class_value]

                # For evaluation
                y_train_unknown_save = y_train_save[y_train == unknown_class_value]
                y_test_unknown_save = y_test_save[y_test == unknown_class_value]
                y_train_unknown = np.array(pd.Series(y_train_unknown_save).astype('category').cat.codes)
                y_test_unknown = np.array(pd.Series(y_test_unknown_save).astype('category').cat.codes)

                # We need the targets to be in {0, ..., C^l} exactly
                classifier_mapper, classifier_ind = np.unique(y_train_known, return_inverse=True)
                classifier_mapping_dict = dict(zip(y_train_known, classifier_ind))

                y_train_known = np.array(list(map(classifier_mapping_dict.get, y_train_known)))
                y_test_known = np.array(list(map(classifier_mapping_dict.get, y_test_known)))

                if model_name == 'ncdsc':
                    test_unknown_spectral_embedding = shuffled_full_spectral_embedding[y_test == unknown_class_value]
                else:
                    test_unknown_spectral_embedding = None
                
                accs, nmis, aris = get_model_performance(model_name, n_runs,
                                                         x_train, y_train, unknown_class_value,
                                                         x_train_known, y_train_known,
                                                         x_train_unknown, y_train_unknown,
                                                         x_test_known, y_test_known,
                                                         x_test_unknown, y_test_unknown,
                                                         test_unknown_spectral_embedding, t)

                accs_avg_2.append(np.mean(accs))
                nmis_avg_2.append(np.mean(nmis))
                aris_avg_2.append(np.mean(aris))
                accs_std_2.append(np.std(accs))
                nmis_std_2.append(np.std(nmis))
                aris_std_2.append(np.std(aris))

            accs_avg_1.append(np.mean(accs_avg_2))
            nmis_avg_1.append(np.mean(nmis_avg_2))
            aris_avg_1.append(np.mean(aris_avg_2))
            accs_std_1.append(np.std(accs_std_2))
            nmis_std_1.append(np.std(nmis_std_2))
            aris_std_1.append(np.std(aris_std_2))
            
    return np.array(accs_avg_1), np.array(nmis_avg_1), np.array(aris_avg_1), np.array(accs_std_1), np.array(nmis_std_1), np.array(aris_std_1)

In [ ]:
def get_perf_for_multiple_models(classes_range, model_names, n_runs , k_folds):
    full_df = pd.DataFrame()
    
    unknown_class_share_list = np.array(classes_range) / len(np.unique(original_y_train))
    
    for model_name in model_names:
        print(f"Getting results for {model_name}...")
        accs_avg, nmis_avg, aris_avg, accs_std, nmis_std, aris_std = get_model_perf_wrt_n_unknown_classes(classes_range=classes_range, model_name=model_name, n_runs=n_runs, k_folds=k_folds)
    
        df = pd.DataFrame({"unknown_class_share_list": unknown_class_share_list, "accs_avg": accs_avg, "accs_std": accs_std, "method": np.repeat(model_name, len(accs_avg))})
        
        full_df = pd.concat([full_df, df])
        full_df.to_csv(f'results_{dataset_name}.csv', index=False)
        
    return full_df

In [ ]:
results_df = get_perf_for_multiple_models(classes_range = range(2, len(np.unique(original_y_train)) - 1),
                                          model_names = ['kmeans', 'ncdkmeans', 'sc', 'ncdsc', 'baseline', 'pbn', 'tabularncd'],  # 'kmeans', 'ncdkmeans', 'sc', 'ncdsc', 'baseline', 'pbn', 'tabularncd'
                                          n_runs = 5, k_folds = 10)

In [ ]:
fig = custom_line_with_error_fill(
    data_frame = results_df,
    x = 'unknown_class_share_list',
    y = 'accs_avg',
    error_y = 'accs_std',
    error_y_mode = 'band', # Either `band` or `bar`.
    color = 'method',
    title = f'Results for {dataset_name} dataset',
    markers = '.',
)
fig.show()

In [ ]:
fig = px.line(results_df, x="unknown_class_share_list", y="accs_avg", color='method', markers='.', title=f'Results for {dataset_name} dataset')
fig.update_layout(xaxis_title="Share of unknown classes", yaxis_title="Average accuracy")
fig.show()

In [ ]:
results_df = pd.read_csv('results_LetterRecognition.csv')
dataset_name = 'LetterRecognition'